In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
%matplotlib inline
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint as sp_randint
from sklearn import metrics

In [0]:
def upload_tmp_file(filename):
    try:
        eider.env.getUploaded(filename, "/tmp/" + filename)
    except Exception as e:
        eider.env.getUploaded("tmp/" + filename, "/tmp/" + filename)

In [0]:
upload_tmp_file("Train.csv")

In [0]:
train= pd.read_csv("/tmp/Train.csv")
#train.columns
#train=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']]

In [0]:
cas=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'casual']]
#plt.hist(cas['casual'])
cas['casual'] = cas['casual'].apply(lambda x: math.log(x + 1))
print("Adding 1 to each column and transforming it to remove skewness")
reg=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'registered']]
reg['registered'] = reg['registered'].apply(lambda x:math.log(x + 1))

In [0]:
def refresh():
    train= pd.read_csv("/tmp/Train.csv")
    train=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']]
    cas=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'casual']]
    cas['casual'] = cas['casual'].apply(lambda x: math.log(x + 1))
    reg=train[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed', 'registered']]
    reg['registered'] = reg['registered'].apply(lambda x:math.log(x + 1))

## Train and Test

In [0]:
X = cas[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed']]
Y = cas['casual']
casX_train,casX_test,casY_train,casY_test=train_test_split(X,Y,test_size=0.3,random_state=123)

In [0]:
X = reg[['season', 'yr','mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp','atemp', 'hum', 'windspeed']]
Y = reg['registered']
regX_train,regX_test,regY_train,regY_test=train_test_split(X,Y,test_size=0.3,random_state=123)

## Regularized Linear Regression

In [0]:
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [0]:
linreg_model = LinearRegression()

In [0]:
linreg_model.fit(casX_train,casY_train)
caslm = linreg_model.predict(casX_test)
metrics.r2_score(casY_test,caslm)

In [0]:
linreg_model.fit(regX_train,regY_train)
reglm = linreg_model.predict(regX_test)
metrics.r2_score(regY_test,reglm)

## Decision Tree Regressor

In [0]:
from sklearn.tree import DecisionTreeRegressor

In [0]:
DTR = DecisionTreeRegressor()
DTR.fit(casX_train,casY_train)

In [0]:
casPred_DTR = DTR.predict(casX_test)
metrics.r2_score(casY_test,casPred_DTR)

In [0]:
DTR.fit(regX_train,regY_train)
regPred_DTR = DTR.predict(regX_test)
metrics.r2_score(regY_test,regPred_DTR)

In [0]:
import graphviz
import os

In [0]:
#dot_data = tree.export_graphviz(DTR, out_file=None)
#graph = graphviz.Source(dot_data)
#graph.render("train_set1")
#os.startfile('train_set1.pdf')

In [0]:
#dot_data = tree.export_graphviz(DTR, out_file=None, max_depth=3,
 #                              feature_names=X_Names,    
 #                        filled=True, rounded=True,  
 #                        special_characters=True)
#graph2 = graphviz.Source(dot_data)
#graph2.render('train_setlinear')

In [0]:
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
         'max_depth':[2,3,4,5,6,7,8]}

In [0]:
DTR = DecisionTreeRegressor()

In [0]:
DTR_GS = GridSearchCV(DTR, param_grid=params)
DTR_GS.fit(casX_train,casY_train)

In [0]:
DTR_GS.best_params_

In [0]:
caspred_DTR_GS = DTR_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_DTR_GS)

In [0]:
DTR_GS = GridSearchCV(DTR, param_grid=params)
DTR_GS.fit(regX_train,regY_train)
print(DTR_GS.best_params_)
regpred_DTR_GS = DTR_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_DTR_GS)

## Random Search

In [0]:
from scipy.stats import randint as sp_randint
 
param_grid2 = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': sp_randint(2,15), 
          'min_samples_leaf': sp_randint(1,11),
         'max_depth':sp_randint(2,8)}

In [0]:
DTR_RS = RandomizedSearchCV(DTR, param_distributions=param_grid2,n_iter=100)
DTR_RS.fit(casX_train,casY_train)
print(DTR_RS.best_params_)
caspred_DTR_RS = DTR_RS.predict(casX_test)
metrics.r2_score(casY_test,caspred_DTR_RS)

In [0]:
DTR_RS.fit(regX_train,regY_train)
print(DTR_RS.best_params_)
regpred_DTR_RS = DTR_RS.predict(regX_test)
metrics.r2_score(regY_test,regpred_DTR_RS)

## Ensemble Models
#### Bagging Regressor

In [0]:
from sklearn.ensemble import BaggingRegressor

In [0]:
baggingR = BaggingRegressor()
baggingR.fit(casX_train,casY_train)
casbag_test_pred = baggingR.predict(casX_test)
metrics.r2_score(casY_test,casbag_test_pred)

In [0]:
baggingR = BaggingRegressor()
baggingR.fit(regX_train,regY_train)
regbag_test_pred = baggingR.predict(regX_test)
metrics.r2_score(regY_test,regbag_test_pred)

## Tuning Hyperparameters Grid Search

In [0]:
params_bag_GS = {"n_estimators": [50,100,200],
              "max_features":[1,2,4,6,8],
              "max_samples": [0.5,0.1],
            "bootstrap": [True, False],
         "bootstrap_features": [True, False]}

In [0]:
Bag_model_GS = GridSearchCV(baggingR, param_grid=params_bag_GS)
Bag_model_GS.fit(casX_train,casY_train)
print(Bag_model_GS.best_params_)
caspred_bag_GS = Bag_model_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_bag_GS)

In [0]:
Bag_model_GS = GridSearchCV(baggingR, param_grid=params_bag_GS)
Bag_model_GS.fit(regX_train,regY_train)
print(Bag_model_GS.best_params_)
regpred_bag_GS = Bag_model_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_bag_GS)

## Random Search

In [0]:
params_bagR_RS = {"n_estimators": sp_randint(50,200),
              "max_features":sp_randint(1,8),
              "max_samples": uniform(0.5,0.1),
            "bootstrap": [True, False],
         "bootstrap_features": [True, False]}

In [0]:
BagR_RS = RandomizedSearchCV(baggingR, param_distributions=params_bagR_RS,n_iter=120)
BagR_RS.fit(casX_train,casY_train)
print(BagR_RS.best_params_)
caspred_bagR_RS = BagR_RS.predict(casX_test)
metrics.r2_score(casY_test,caspred_bagR_RS)

In [0]:
BagR_RS.fit(regX_train,regY_train)
print(BagR_RS.best_params_)
regpred_bagR_RS = BagR_RS.predict(regX_test)
metrics.r2_score(regY_test,regpred_bagR_RS)

In [0]:
##Random Forest Regressor

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
rfr = RandomForestRegressor()
params_RF = {"max_depth": [3,5,6,7,8,9],
              "max_features":['auto', 'sqrt', 'log2'],
              "min_samples_split": [2, 3,5,7],
              "min_samples_leaf": [1, 3,5,6]}

In [0]:
model_RF_GS = GridSearchCV(rfr, param_grid=params_RF)
model_RF_GS.fit(casX_train,casY_train)
print(model_RF_GS.best_params_)
caspred_RF_GS = model_RF_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_RF_GS)

In [0]:
model_RF_GS.fit(regX_train,regY_train)
print(model_RF_GS.best_params_)
regpred_RF_GS = model_RF_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_RF_GS)

## Random Search

In [0]:
params_RF_RS = {"max_depth": sp_randint(3,9),
              "max_features":['auto', 'sqrt', 'log2'],
              "min_samples_split":sp_randint(2,7),
              "min_samples_leaf": sp_randint(1,6)}

In [0]:
rfr_RS = RandomizedSearchCV(rfr, param_distributions=params_RF_RS,n_iter=100)
rfr_RS.fit(casX_train,casY_train)
print(rfr_RS.best_params_)
caspred_RF_RS = rfr_RS.predict(casX_test)
metrics.r2_score(casY_test,caspred_RF_RS)

In [0]:
rfr_RS.fit(regX_train,regY_train)
print(rfr_RS.best_params_)
regpred_RF_RS = rfr_RS.predict(regX_test)
metrics.r2_score(regY_test,regpred_RF_RS)

# AdaBoost Regressor

In [0]:
from sklearn.ensemble import AdaBoostRegressor

## Grid Search

In [0]:
params_AdbR_GS = {'learning_rate':[0.05,0.1,0.2,0.6,0.8,1],
        'n_estimators': [50,60,100],
                 'loss' : ['linear', 'square', 'exponential']}

In [0]:
model_AdaR_GS = GridSearchCV(AdaBoostRegressor(), param_grid=params_AdbR_GS)
model_AdaR_GS.fit(casX_train,casY_train)
model_AdaR_GS.best_params_
caspred_AdaR_GS = model_AdaR_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_AdaR_GS)

In [0]:
model_AdaR_GS.fit(regX_train,regY_train)
model_AdaR_GS.best_params_
regpred_AdaR_GS = model_AdaR_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_AdaR_GS)

# Gradient Boosting Regressor

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

### Tuning Hyperparameters
#### Grid Search

In [0]:
params_GBR_GS = {"max_depth": [3,5,6,7],
              "max_features":['auto', 'sqrt', 'log2'],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
            'learning_rate':[0.05,0.1,0.2],
            'n_estimators': [10,30,50,70]}

In [0]:
model_GradR2_GS = GridSearchCV(GradientBoostingRegressor(), param_grid=params_GBR_GS)
model_GradR2_GS.fit(casX_train,casY_train)
print(model_GradR2_GS.best_params_)
caspred_GradR_GS = model_GradR2_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_GradR_GS)

In [0]:
model_GradR2_GS.fit(regX_train,regY_train)
print(model_GradR2_GS.best_params_)
regpred_GradR_GS = model_GradR2_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_GradR_GS)

# XgBoost Regressor
## Tuning Hyperparameters
### Grid Search

In [0]:
from xgboost import XGBRegressor

In [0]:
params_xgbR_GS = {"max_depth": [3,4,5,6,7,8],
              "min_child_weight" : [3,4,5,6,7,8,9,10,11,12,13,14,15],
            'learning_rate':[0.05,0.1,0.2,0.25,0.8,1],
            'n_estimators': [100,200,300,400,500]}

In [0]:
model_xgbR_GS = GridSearchCV(XGBRegressor(), param_grid=params_xgbR_GS)

In [0]:

model_xgbR_GS.fit(casX_train,casY_train)
print(model_xgbR_GS.best_params_)
caspred_xgbR_GS = model_xgbR_GS.predict(casX_test)
metrics.r2_score(casY_test,caspred_xgbR_GS)

In [0]:
model_xgbR_GS.fit(regX_train,regY_train)
print(model_xgbR_GS.best_params_)
regpred_xgbR_GS = model_xgbR_GS.predict(regX_test)
metrics.r2_score(regY_test,regpred_xgbR_GS)